# QrackCircuit

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
width = 12

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

import os

# Change these according to your OpenCL devices and system specifications:
os.environ['QRACK_MAX_PAGE_QB']='27'
os.environ['QRACK_QUNITMULTI_DEVICES']='1'
os.environ['QRACK_QPAGER_DEVICES']='1'
os.environ['QRACK_MAX_ALLOC_MB']='147456,15872'
os.environ['QRACK_MAX_PAGING_QB']='30'
os.environ['QRACK_MAX_CPU_QB']='34'

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [3]:
import math
import random

sqrt1_2 = 1 / math.sqrt(2)

def x_to_y(circ, q):
    circ.mtrx([1, 0, 0, 1j], q)
    return 1

def x_to_z(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def y_to_z(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 2

def y_to_x(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    return 1

def z_to_x(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def z_to_y(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    circ.mtrx([1, 0, 0, 1j], q)
    return 2

def cx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 1)
    return 1

def cy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 1)
    return 1

def cz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    return 1

def acx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 0)
    return 1

def acy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 0)
    return 1

def acz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 1

def swap(circ, q1, q2):
    circ.swap(q1, q2)
    return 1

def nswap(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    circ.swap(q1, q2)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 3

def pswap(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    circ.swap(q1, q2)
    return 2

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 2

def iswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    circ.mtrx([1, 0, 0, 1j], q1)
    circ.mtrx([1, 0, 0, 1j], q2)
    return 4

def iiswap(circ, q1, q2):
    circ.mtrx([1, 0, 0, -1j], q2)
    circ.mtrx([1, 0, 0, -1j], q1)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    circ.swap(q1, q2)
    return 4

def random_circuit(width, circ):
    gate_count = 0
    
    single_bit_gates = { 0: (z_to_x, z_to_y), 1: (x_to_y, x_to_z), 2: (y_to_z, y_to_x) } 
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))

    # Don't repeat bases:
    bases = [0] * width
    directions = [0] * width
    
    for i in range(3 * width):
        # Single bit gates
        for j in range(width):
            # Reset basis, every third layer
            if i % 3 == 0:
                bases[j] = random.randint(0, 2)
                directions[j] = random.randint(0, 1)
            
            # Sequential basis switch
            gate = single_bit_gates[bases[j]][directions[j]]
            gate_count += gate(circ, j)

            # Cycle through all 3 Pauli axes, every 3 layers
            if directions[j]:
                bases[j] -= 1
                if bases[j] < 0:
                    bases[j] += 3
            else:
                bases[j] += 1
                if bases[j] > 2:
                    bases[j] -= 3
                
            # Rotate around local Z axis
            rnd = random.uniform(0, 2 * math.pi)
            circ.mtrx([1, 0, 0, math.cos(rnd) + math.sin(rnd) * 1j], j)
            gate_count += 1
            
        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(row_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                g = random.choice(two_bit_gates)
                gate_count += g(circ, b1, b2)

    print("Gate count (before optimization): " + str(gate_count))

    return circ

In [4]:
from pyqrack import QrackSimulator, QrackCircuit

qcircuit = QrackCircuit()
random_circuit(width, qcircuit)
qcircuit.out_to_file('qrack_circuit.qc')

data = ''
with open('qrack_circuit.qc', 'r') as file:
    data = file.read().split()
    
print("Gate count (after optimization): " + str(data[1]))

Gate count (before optimization): 1251
Gate count (after optimization): 385


In [5]:
import time

start = time.perf_counter()
qcircuit = QrackCircuit()
qcircuit.in_from_file('qrack_circuit.qc')
qsim = QrackSimulator(width)
qcircuit.run(qsim)
print("Qubits: ", qsim._qubitCount)
print("Measurement result: ", qsim.m_all())
print("Execution seconds: " + str(time.perf_counter() - start))


Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Qubits:  12
Measurement result:  2921
Execution seconds: 0.170096564999767


If we have Qiskit and numpy installed, we can convert to a Qiskit circuit.

In [6]:
circ = QrackCircuit.file_to_qiskit_circuit('qrack_circuit.qc')
print(circ)

                                                                     »
 q_0: ───────────────────────────────────────────────────────────────»
                                                     ┌──────────────┐»
 q_1: ───────────────────────────────────────────────┤0             ├»
                                     ┌──────────────┐│              │»
 q_2: ───────────────────────────────┤0             ├┤              ├»
                     ┌──────────────┐│              ││              │»
 q_3: ───────────────┤0             ├┤              ├┤  Multiplexer ├»
      ┌─────────────┐│              ││              ││              │»
 q_4: ┤ Multiplexer ├┤              ├┤  Multiplexer ├┤              ├»
      ├─────────────┤│              ││              ││              │»
 q_5: ┤ Multiplexer ├┤  Multiplexer ├┤              ├┤1             ├»
      ├─────────────┤│              ││              │└──────────────┘»
 q_6: ┤ Multiplexer ├┤              ├┤1             ├────────────────»
      

In [7]:
from qiskit.compiler.transpiler import transpile

basis_gates = ["u", "cx"]
circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)
print(circ)

global phase: 5.0261
            ┌─────────────────┐            ┌───────────────────────────┐»
 q_0: ──────┤ U(π/2,π/2,-π/2) ├─────────■──┤ U(1.5955,0.65113,-1.3833) ├»
           ┌┴─────────────────┴─┐       │  └───────────────────────────┘»
 q_1: ─────┤ U(π,2.5604,-2.152) ├───────┼────────────────■──────────────»
           ├────────────────────┤       │                │              »
 q_2: ─────┤ U(π,-1.8925,1.249) ├───────┼────────────────┼──────────────»
          ┌┴────────────────────┤       │                │              »
 q_3: ────┤ U(π,2.5604,0.98962) ├───────┼────────────────┼──────────────»
          ├─────────────────────┤     ┌─┴─┐              │              »
 q_4: ────┤ U(2.0642,-π/2,-π/2) ├─────┤ X ├──────────────┼──────────────»
        ┌─┴─────────────────────┴─┐   └───┘            ┌─┴─┐            »
 q_5: ──┤ U(0.72971,-π/2,-2.3397) ├────────────────────┤ X ├────────────»
        └─┬─────────────────────┬─┘                    └───┘            »
 q_6: ────┤ U(0.8

If the basis gates can be converted, a Qiskit `QuantumCircuit` can also be read into a `QrackCircuit`.

In [8]:
qcircuit = QrackCircuit()
qcircuit.in_from_qiskit_circuit(circ)
qcircuit.out_to_file('qiskit.qc')

In [9]:
import quimb as qu
import quimb.tensor as qtn

tcirc = qtn.Circuit(circ.num_qubits)

for gate in circ.data:
    o = gate.operation
    if o.name == "u":
        th = float(o.params[0])
        ph = float(o.params[1])
        lm = float(o.params[2])

        tcirc.apply_gate('U3', th, ph, lm, gate.qubits[0].index)
    else:
        tcirc.apply_gate('CNOT', gate.qubits[1].index, gate.qubits[0].index)

tcirc.amplitude('01' * (width // 2))

/tmp/ipykernel_36699/3029626155.py:13: DeprecationWarning: 'Bit.index' is deprecated since Qiskit Terra 0.17 and will be removed in a future release. Bits may be in more than one register. Use 'QuantumCircuit.find_bit' to find all the containing registers within a circuit, and the index of the bit within the circuit.
  tcirc.apply_gate('U3', th, ph, lm, gate.qubits[0].index)
/tmp/ipykernel_36699/3029626155.py:15: DeprecationWarning: 'Bit.index' is deprecated since Qiskit Terra 0.17 and will be removed in a future release. Bits may be in more than one register. Use 'QuantumCircuit.find_bit' to find all the containing registers within a circuit, and the index of the bit within the circuit.
  tcirc.apply_gate('CNOT', gate.qubits[1].index, gate.qubits[0].index)


(2.4079459742330398e-05+0.004180523745630467j)